In [1]:
#who dat boii?

#load packages
import numpy as np
import sklearn.model_selection
import matplotlib.pyplot as plt

In [ ]:
#make helper functions

def plot_digit(X, y, idx):
    img = X[idx].reshape(28,28)
    plt.imshow(img, cmap='Greys',  interpolation='nearest')
    plt.title('true label: %d' % y[idx,0])
    plt.show()

In [ ]:
#load data

from mlxtend.data import mnist_data
X, y = mnist_data()


#we just focus on 1's and 0's:
keep=(y==0) | (y==1)
X=X[keep,:]
y=y[keep]

y=np.expand_dims(y,1)

In [ ]:
#inspect data
rng = np.random.default_rng()
max_dim_y = np.max(np.shape(y))
idx = rng.integers(max_dim_y)
#(plot digit)
plot_digit(X,y,idx)

In [ ]:
# create a loss function:
def avrLoss(y,a):
    loss = -(y*np.log(a)+(1-y)*np.log(1-a))
    
    return np.mean(loss)

In [ ]:
#test loss function:
#(just run this cell and see what happens)


loss=avrLoss(np.array([0,1,0,1]),np.linspace(.1,.9,4))
assert np.all(loss==0.5543313122608056)
#no error means the results match

In [ ]:
#create a sigmoid
def sigmoid(z):
    f = 1/(1+np.exp(-z))
    return f
    #INSERT CODE

In [ ]:
#check sigmoid
#(just run this cell and see what happens)

out=sigmoid(np.array(range(-5,5)))
assert np.all(np.around(out,8)==np.array([0.00669285, 0.01798621, 0.04742587, 0.11920292, 0.26894142, 0.5, 0.73105858, 0.88079708, 0.95257413, 0.98201379]))
#no error means the results match

In [ ]:
#make test & train data

Xtrain,Xtest,ytrain,ytest=sklearn.model_selection.train_test_split(X,y,test_size=0.3,shuffle=True)

In [ ]:
#train

#INITIATE WEIGHTS AND BIASES (w,b)
[M,N] = np.shape(Xtrain)
w=np.reshape(np.zeros(N),[N,1])
b=0

nEpoch=60

accuracy=np.zeros(nEpoch)
trainRate=0.05#pick a number less than 1

for iTrain in range(nEpoch):
    #evaluate
    z = np.matmul(Xtrain,w)+b
    z = np.reshape(z,[M,1])
    #CALCULATE a
    a = sigmoid(z)

    #calculate loss & accuracy
    loss=avrLoss(ytrain,a)

    yhat=np.where(a > 0.5, 1, 0)
    accuracy[iTrain]=np.mean(yhat==ytrain)


    #update weights
    temp = np.reshape(np.sum((a-ytrain)*Xtrain,0),[N,1])
    wgrad=(1/M)*temp
    bgrad= (1/M)*np.sum((a-ytrain))

    assert(wgrad.shape == w.shape)

    #UPDATE W AND b
    w = w - trainRate*wgrad
    b = b - trainRate*bgrad



    if np.mod(iTrain,10)==0:
        print('Epoch: ',iTrain)
        print('loss = ',loss)
        print('accuracy',accuracy[iTrain])

Y_prediction_test = sigmoid(np.matmul(Xtest,w)+b)>0.5
Y_prediction_train = sigmoid(np.matmul(Xtrain,w)+b)>0.5

print("train accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_train - ytrain)) * 100))
print("test accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_test - ytest)) * 100))

In [ ]:
#see what the network does:

img = w.reshape(28,28)
h=plt.imshow(img, cmap='Greys',  interpolation='nearest')
plt.title('My Network')
plt.colorbar(h)
plt.show()

In [ ]:
#commpare with scikit-learn:

from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0).fit(Xtrain, np.ravel(ytrain))

clf.predict(X)
print('Train accuracy: ',clf.score(Xtrain, ytrain))
print('Test accuracy: ',clf.score(Xtest, ytest))

img = (clf.coef_).reshape(28,28)
plt.imshow(img, cmap='Greys',  interpolation='nearest')
plt.title('sklearn Network')
plt.show()